<a href="https://colab.research.google.com/github/trie0000/external/blob/main/code_20260106_1505.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""DL_Basic_2025_Competition_NYUv2_baseline.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/17t7uAU0aST5aUt6sIJCqFneGlQrxFrJY

# Deep Learning 基礎講座　最終課題: NYUv2 セマンティックセグメンテーションcoarse_w=0」

## 概要
RGB画像から、画像内の各ピクセルがどのクラスに属するかを予測するセマンティックセグメンテーションタスク.

### データセット
- データセット: NYUv2 dataset
- 訓練データ: 795枚
- テストデータ: 654枚
- 入力: RGB画像 + 深度マップ（元画像サイズは可変）
- 出力: 13クラスのセグメンテーションマップ
- 評価指標: Mean IoU (Intersection over Union)

### データセットの詳細（[NYU Depth Dataset V2](https://cs.nyu.edu/~fergus/datasets/nyu_depth_v2.html)）
- 画像は屋内シーンを撮影したもので、家具や壁、床などの物体が含まれています.
- 各画像に対して13クラスのセグメンテーションラベルが提供されます.
- データは以下のディレクトリ構造で提供:
```
data/NYUv2/
├─train/
│  ├─image/      # RGB画像
│  │    000000.png
│  │    ...
│  │
│  ├─depth/      # 深度マップ
│  │    000000.png
│  │    ...
│  │
│  └─label/      # 13クラスセグメンテーション（教師ラベル）
│       000000.png
│       ...
└─test/
   ├─image/      # RGB画像
   │    000000.png
   │    ...
   │  ├─depth/   # 深度マップ
   │    000000.png
   │    ...
```

### タスクの詳細
- 入力のRGB画像と深度マップから、各ピクセルが13クラスのどれに属するかを予測するタスクです.
- 評価はMean IoUを使用します．
  - 各クラスごとにIoUを計算し、その平均を取ります.
  - IoUは以下の式で計算:
  $$IoU = \frac{TP}{TP + FP + FN}$$
    - TP: True Positive（正しく予測されたピクセル数）
    - FP: False Positive（誤って予測されたピクセル数）
    - FN: False Negative（見逃したピクセル数）

### 前処理
- 入力画像は512×512にリサイズされます.
- ピクセル値は0-1に正規化されます.
- セグメンテーションラベルは0-12の整数値（13クラス）です．
  - 255はignore index（評価から除外）

### 提出形式
- テスト画像（RGB + Depth）の各ピクセルに対してクラス（0~12）を予測したものをnumpy配列として保存されます.
- ファイル名: `submission.npy`
- 配列の形状: [テストデータ数, 高さ, 幅]
- 各ピクセルの値: 0-12の整数（予測クラス）

## 考えられる工夫の例
- 事前学習モデルの fine-tuning
    - ImageNetなどで事前学習されたモデルを本データセットでfine-tuningすることで性能向上が見込めます.
- 損失関数の再設計
    - クラスごとの出現頻度に応じて損失を補正するように損失関数を設計すると、クラス分布の不均衡に対してロバストな学習ができます.
- 画像の前処理
    - RandomResizedCrop / Flip / ColorJitter 等のデータ拡張を追加することで，汎化性能の向上が見込めます．

## 修了要件を満たす条件
- ベースラインでは，omnicampus 上での性能評価において， 38.2% となります．したがって，ベースラインである 38.2% を超えた提出のみ，修了要件として認めます．
- ベースラインから改善を加えることで， 50%以上に性能向上することを運営で確認しています．こちらを 1つの指標として取り組んでみてください．

## 注意点
- 最終的な予測モデルは，**配布している訓練データを用いて学習**（ファインチューニング含む）したものとしてください．
- 学習を行わず，**事前学習済みモデルの知識のみを利用した推論は禁止**します．
（例: ChatGPT 等の LLM に入力して推論を得るのみ）

### 事前学習モデルの利用
許可される事項
- **構成要素としての事前学習モデルの利用**: 自身で実装したアーキテクチャの一部（特徴抽出，埋め込みなど）として事前学習モデル（BERT，ViT など）を利用することは可能です．
- **ファインチューニング**: 上記の用途で利用している事前学習モデルのファインチューニングは可能です．

禁止される事項
- **タスク解決用の事前学習モデルの利用**: transformers などで提供されている，対象タスクを直接解くための事前学習モデルでそのまま推論のみ，またはファインチューニングのみで利用することは禁止とします．
  - 禁止事項の例: VQA タスクを直接解くための事前学習モデルを VQA タスクで利用する．

### データの準備
データをダウンロードした際に，google drive したため，利用するために google drive をマウントする必要があります．また， drive 上で展開することができないため，/content ディレクトリ下にコピーし "data.zip" を展開します．
google drive 上に "data.zip" が配置されていない場合は実行できません．google drive 上に "data.zip" (**831MB**) を配置することが可能であれば，"data_download.ipynb" を先に実行してください．難しい場合は，omnicampus 演習環境を利用してください．．
"""

# omnicampus 上では 4 セル目まで実行不要
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

# データダウンロード用の notebook にてgoogle drive への保存後，
# 反映に時間がかかる可能性がありますので，google drive のマウント後，
# data.zip がディレクトリ内にあることを確認してから実行してください．
# data.zip を /content 下にコピーする
!cp "/content/drive/MyDrive/data.zip" "/content"

# Commented out IPython magic to ensure Python compatibility.
# カレントディレクトリ下のファイル群を確認
# data.zip が表示されれば問題ないです
# %ls

# データを解凍する
!unzip data.zip
!mkdir data
!mv train test data/

"""omnicampus 演習環境では，data_download.ipynb のマウント，zip 化，drive へのコピーを実行しないことで，"data.zip" を解凍した形で配置されます．したがって，data ディレクトリが存在するディレクトリをカレントディレクトリとするだけで良いです．


"""

# Commented out IPython magic to ensure Python compatibility.
# omnicampus 実行用
# 以下の例では/workspace/Segmentation/split_data_scripts/omnicampus に data ディレクトリがあると想定
# %cd /workspace/Segmentation/split_data_scripts_omnicampus

# omnicampus 実行用
!pip install h5py scikit-image

"""# import library"""

Mounted at /content/drive
Archive:  data.zip
  inflating: data/train/image/000600.png  
  inflating: data/train/image/000320.png  
  inflating: data/train/image/000491.png  
  inflating: data/train/image/000502.png  
  inflating: data/train/image/000129.png  
  inflating: data/train/image/000044.png  
  inflating: data/train/image/000652.png  
  inflating: data/train/image/000919.png  
  inflating: data/train/image/000528.png  
  inflating: data/train/image/000853.png  
  inflating: data/train/image/000177.png  
  inflating: data/train/image/000584.png  
  inflating: data/train/image/001319.png  
  inflating: data/train/image/000597.png  
  inflating: data/train/image/000223.png  
  inflating: data/train/image/001350.png  
  inflating: data/train/image/000404.png  
  inflating: data/train/image/000488.png  
  inflating: data/train/image/000268.png  
  inflating: data/train/image/000481.png  
  inflating: data/train/image/000341.png  
  inflating: data/train/image/000159.png  
  inflati

'# import library'

In [5]:
# =========================
# Cell 1: Common Setup (Logger Enhanced)
# =========================
import os
import time
import json
import random
import shutil
import logging
import numpy as np
from zipfile import ZipFile, ZIP_DEFLATED
from PIL import Image
from tqdm import tqdm
import cv2
import albumentations as A

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import models
from torch.amp import autocast, GradScaler

# 環境設定
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# --- Constants ---
CLASS_NAMES = ["Bed", "Book", "Ceiling", "Chair", "Floor", "Cabinet", "Object", "Picture",
               "Sofa", "Desk", "TV", "Wall", "Window"]
NUM_CLASSES = 13
IGNORE_INDEX = 255

# Target IDs
BOOK_ID = 1
CABINET_ID = 5
OBJECT_ID = 6
DESK_ID = 9

# --- Utils ---
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def now_ts():
    return time.strftime("%Y%m%d_%H%M%S")

def make_run_id(prefix: str):
    return f"{prefix}_{now_ts()}"

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)
    return path

def write_json(path: str, obj: dict):
    with open(path, "w") as f:
        json.dump(obj, f, indent=2)

def append_jsonl(path: str, obj: dict):
    with open(path, "a") as f:
        f.write(json.dumps(obj) + "\n")

# --- Drive Utils ---
def mount_drive():
    try:
        from google.colab import drive
        drive.mount("/content/drive")
    except Exception as e:
        print(f"[WARN] Drive mount skipped or failed: {e}")

def copy_to_drive_if_needed(src_path: str, dst_dir: str):
    if dst_dir is None: return
    ensure_dir(dst_dir)
    if os.path.isdir(src_path):
        raise RuntimeError("Directory copy is disabled.")
    shutil.copy2(src_path, os.path.join(dst_dir, os.path.basename(src_path)))

def zip_submission_only(npy_path: str, zip_path: str):
    print(f"[LONG] Start zipping submission only: {zip_path}")
    with ZipFile(zip_path, "w", ZIP_DEFLATED) as zf:
        zf.write(npy_path, arcname=os.path.basename(npy_path))
    print(f"[DONE] submission zip: {zip_path}")
    return zip_path

# --- Logger & Metrics Formatting ---
class RunLogger:
    def __init__(self, run_dir: str, run_id: str):
        self.run_dir = run_dir
        self.run_id = run_id
        self.logs_dir = ensure_dir(os.path.join(run_dir, "logs"))
        self.cfg_path = os.path.join(self.logs_dir, "config.json")
        self.train_log = os.path.join(self.logs_dir, "train_log.jsonl")
        self.val_log = os.path.join(self.logs_dir, "val_log.jsonl")
        self.batch_log = os.path.join(self.logs_dir, "batch_log.jsonl")
        self.batch_buf = []

    def save_config(self, cfg: dict):
        write_json(self.cfg_path, cfg)

    def log_batch(self, tag: str, epoch: int, batch: int, losses: dict):
        self.batch_buf.append({
            "run_id": self.run_id, "tag": tag, "epoch": epoch, "batch": batch,
            "losses": {k: float(v) for k, v in losses.items()}
        })

    def flush_batch(self):
        if not self.batch_buf: return
        with open(self.batch_log, "a") as f:
            for e in self.batch_buf: f.write(json.dumps(e) + "\n")
        self.batch_buf = []

    def log_epoch_train(self, tag: str, epoch: int, lr: float, avg_losses: dict):
        append_jsonl(self.train_log, {
            "run_id": self.run_id, "tag": tag, "epoch": epoch, "lr": float(lr),
            "losses": {k: float(v) for k, v in avg_losses.items()}
        })

    def log_epoch_val(self, tag: str, epoch: int, metrics: dict, names: list, cm_np=None):
        append_jsonl(self.val_log, {
            "run_id": self.run_id, "tag": tag, "epoch": epoch,
            "miou": float(metrics["miou"]),
            "class_iou": {names[i]: float(metrics["class_iou"][i]) for i in range(len(names))},
            "class_precision": {names[i]: float(metrics["class_precision"][i]) for i in range(len(names))},
            "class_recall": {names[i]: float(metrics["class_recall"][i]) for i in range(len(names))}
        })
        # ★ここが重要: 混合行列(Numpy配列)を保存する処理
        if cm_np is not None:
            np.save(os.path.join(self.logs_dir, f"confusion_matrix_{tag}_epoch_{epoch}.npy"), cm_np)

def setup_console_logger(run_dir):
    """Boost学習用の簡易ロガー"""
    logger = logging.getLogger(f"train_logger_{run_dir}")
    logger.setLevel(logging.INFO)
    if logger.hasHandlers(): logger.handlers.clear()
    logger.propagate = False
    formatter = logging.Formatter('[%(asctime)s] %(message)s')
    file_path = os.path.join(run_dir, "log.txt")
    file_handler = logging.FileHandler(file_path); file_handler.setFormatter(formatter)
    stream_handler = logging.StreamHandler(); stream_handler.setFormatter(formatter)
    logger.addHandler(file_handler); logger.addHandler(stream_handler)
    print(f"[OUTPUT] Log file created at: {file_path}")
    return logger

# --- Metrics ---
def compute_metrics_from_cm(cm: torch.Tensor):
    cm = cm.float()
    tp = torch.diag(cm)
    fp = cm.sum(dim=0) - tp
    fn = cm.sum(dim=1) - tp
    iou = tp / (tp + fp + fn + 1e-8)
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    return {
        "miou": iou.mean().item(), "class_iou": iou.cpu().tolist(),
        "class_precision": precision.cpu().tolist(), "class_recall": recall.cpu().tolist(),
    }

def update_cm(cm: torch.Tensor, pred: torch.Tensor, target: torch.Tensor, n_classes: int, ignore_index: int):
    pred = pred.detach().view(-1).to("cpu")
    target = target.detach().view(-1).to("cpu")
    mask = (target != ignore_index)
    cm += torch.bincount(target[mask] * n_classes + pred[mask], minlength=n_classes**2).view(n_classes, n_classes)
    return cm

# ★追加: コンソールに見やすく表示するための関数
def fmt_metrics_console(tag: str, epoch: int, miou: float, class_iou, class_precision, class_recall, names):
    print(f"[{tag}] Epoch {epoch:02d} | mIoU={miou:.5f}")
    # ヘッダー
    header = f"{'Class':<10} {'IoU':>8} {'Prec':>8} {'Rec':>8}"
    print(header)
    print("-" * len(header))
    # 各クラスのスコア
    for i, name in enumerate(names):
        print(f"{name:<10} {class_iou[i]:>8.3f} {class_precision[i]:>8.3f} {class_recall[i]:>8.3f}")

class IoUMeter:
    """Boost学習用 (Numpy Base)"""
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.reset()
    def reset(self):
        self.confusion_matrix = np.zeros((self.num_classes, self.num_classes), dtype=np.int64)
    def update(self, pred, target):
        if isinstance(pred, torch.Tensor): pred = pred.cpu().numpy()
        if isinstance(target, torch.Tensor): target = target.cpu().numpy()
        pred = pred.flatten(); target = target.flatten()
        mask = (target != IGNORE_INDEX)
        valid_indices = target[mask] * self.num_classes + pred[mask]
        self.confusion_matrix += np.bincount(valid_indices, minlength=self.num_classes**2).reshape(self.num_classes, self.num_classes)
    def get_metrics(self):
        cm = self.confusion_matrix
        tp = np.diag(cm)
        fp = cm.sum(axis=0) - tp; fn = cm.sum(axis=1) - tp
        iou = tp / (tp + fp + fn + 1e-10)
        precision = tp / (tp + fp + 1e-10)
        recall = tp / (tp + fn + 1e-10)
        return {"iou": iou, "miou": np.nanmean(iou), "precision": precision, "recall": recall, "cm": cm}

# --- Fusion Logic (Unified) ---
def fuse_conditional_boost(base_logits, boost_logits, alpha=2.0):
    TARGET_IDS = [BOOK_ID, CABINET_ID, OBJECT_ID, DESK_ID]
    base_probs = F.softmax(base_logits, dim=1)
    base_pred = base_probs.argmax(dim=1)
    mask = torch.zeros_like(base_pred, dtype=torch.bool)
    for tid in TARGET_IDS:
        mask |= (base_pred == tid)
    final_logits = base_logits.clone()
    boost_book_score = boost_logits[:, BOOK_ID, :, :]
    final_logits[:, BOOK_ID, :, :][mask] += boost_book_score[mask] * alpha
    return F.softmax(final_logits, dim=1)

# --- Dataset ---
def estimate_height_from_depth(depth_np: np.ndarray) -> np.ndarray:
    H, W = depth_np.shape
    y_grid = np.linspace(0, 1, H).reshape(H, 1).repeat(W, axis=1).astype(np.float32)
    height_map = y_grid * depth_np
    max_val = float(height_map.max())
    if max_val > 0: height_map /= max_val
    return height_map.astype(np.float32)

class NYUv2Dataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None, return_label=True):
        self.split = split
        self.transform = transform
        self.return_label = return_label
        src_split = 'train' if split in ['train', 'val'] else 'test'
        self.images_dir = os.path.join(root_dir, src_split, 'image')
        self.depths_dir = os.path.join(root_dir, src_split, 'depth')
        self.labels_dir = os.path.join(root_dir, src_split, 'label') if src_split == 'train' else None
        self.filenames = sorted([f for f in os.listdir(self.images_dir) if f.endswith('.png')])
        self.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
        self.std  = np.array([0.229, 0.224, 0.225], dtype=np.float32)
        self.d_mean, self.d_std = 0.5, 0.25

    def __len__(self): return len(self.filenames)

    def __getitem__(self, idx):
        fname = self.filenames[idx]
        rgb = np.array(Image.open(os.path.join(self.images_dir, fname)).convert('RGB'))
        depth = np.array(Image.open(os.path.join(self.depths_dir, fname)))
        if depth.ndim == 3: depth = depth[:, :, 0]
        depth = depth.astype(np.float32) / (65535.0 if depth.max() > 255 else 255.0)
        h_map = estimate_height_from_depth(depth)

        if self.labels_dir and self.return_label:
            label = np.array(Image.open(os.path.join(self.labels_dir, fname)))
        else:
            label = np.zeros(depth.shape, dtype=np.int32)

        if self.transform:
            augmented = self.transform(image=rgb, depth=depth, height=h_map, mask=label)
            rgb, depth, h_map, label = augmented['image'], augmented['depth'], augmented['height'], augmented['mask']

        rgb = (rgb.astype(np.float32) / 255.0 - self.mean) / self.std
        depth = (depth - self.d_mean) / self.d_std
        h_map = (h_map - self.d_mean) / self.d_std

        x = torch.cat([
            torch.from_numpy(rgb.transpose(2, 0, 1)).float(),
            torch.from_numpy(depth).unsqueeze(0).float(),
            torch.from_numpy(h_map).unsqueeze(0).float()
        ], dim=0) # 5ch

        if self.split == 'test': return x, fname
        return x, torch.from_numpy(label).long()

In [19]:
# =========================
# Cell 2: Base Model (Consistent Split)
# =========================

# --- Architecture Components ---
class SeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, dilation=1, bias=False):
        super().__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size, stride, padding, dilation, groups=in_channels, bias=bias)
        self.pointwise = nn.Conv2d(in_channels, out_channels, 1, 1, 0, 1, 1, bias=bias)
    def forward(self, x): return self.pointwise(self.depthwise(x))

class ConvBNReLU(nn.Module):
    def __init__(self, in_ch, out_ch, k=3, p=1, d=1):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, k, padding=p, dilation=d, bias=False),
            nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.block(x)

class ASPP(nn.Module):
    def __init__(self, in_ch, out_ch=256, rates=(12, 24, 36)):
        super().__init__()
        r1, r2, r3 = rates
        self.b1 = ConvBNReLU(in_ch, out_ch, 1, 0, 1)
        self.b2 = ConvBNReLU(in_ch, out_ch, 3, r1, r1)
        self.b3 = ConvBNReLU(in_ch, out_ch, 3, r2, r2)
        self.b4 = ConvBNReLU(in_ch, out_ch, 3, r3, r3)
        self.b5 = nn.Sequential(nn.AdaptiveAvgPool2d(1), ConvBNReLU(in_ch, out_ch, 1, 0, 1))
        self.proj = nn.Sequential(ConvBNReLU(out_ch * 5, out_ch, 1, 0, 1), nn.Dropout(0.1))
    def forward(self, x):
        h, w = x.shape[2:]
        return self.proj(torch.cat([
            self.b1(x), self.b2(x), self.b3(x), self.b4(x),
            F.interpolate(self.b5(x), size=(h, w), mode='bilinear', align_corners=False)
        ], dim=1))

class ResNeXtDeepLabV3Plus_OS8(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, in_channels=5, aspp_rates=(12,24,36)):
        super().__init__()
        backbone = models.resnext101_32x8d(
            weights=models.ResNeXt101_32X8D_Weights.IMAGENET1K_V1,
            replace_stride_with_dilation=[False, True, True],
        )
        old_conv = backbone.conv1
        new_conv = nn.Conv2d(in_channels, old_conv.out_channels, 7, stride=2, padding=3, bias=False)
        with torch.no_grad():
            new_conv.weight[:, :3] = old_conv.weight
            new_conv.weight[:, 3:] = old_conv.weight.mean(dim=1, keepdim=True).repeat(1, in_channels-3, 1, 1)

        self.enc0 = nn.Sequential(new_conv, backbone.bn1, backbone.relu)
        self.pool = backbone.maxpool
        self.enc1 = backbone.layer1
        self.enc2 = backbone.layer2
        self.enc3 = backbone.layer3
        self.enc4 = backbone.layer4

        self.aspp = ASPP(2048, 256, rates=aspp_rates)
        self.low_proj = nn.Sequential(nn.Conv2d(256, 48, 1, bias=False), nn.BatchNorm2d(48), nn.ReLU(inplace=True))
        self.dec1 = ConvBNReLU(256 + 48, 256, 3, 1, 1)
        self.dec2 = ConvBNReLU(256, 256, 3, 1, 1)
        self.seg_head = nn.Conv2d(256, num_classes, 1)
        self.boundary_head = nn.Conv2d(256, 1, 1)
        self.aux_head = nn.Sequential(ConvBNReLU(1024, 256, 3, 1, 1), nn.Dropout(0.1), nn.Conv2d(256, num_classes, 1))

    def forward(self, x, return_aux=False, return_boundary=False):
        H, W = x.shape[2:]
        x0 = self.enc0(x); x1 = self.pool(x0)
        low = self.enc1(x1); x2 = self.enc2(low)
        mid = self.enc3(x2); x4 = self.enc4(mid)

        xA = self.aspp(x4)
        xA = F.interpolate(xA, size=low.shape[2:], mode='bilinear', align_corners=False)
        feat = self.dec2(self.dec1(torch.cat([xA, self.low_proj(low)], dim=1)))

        outs = [F.interpolate(self.seg_head(feat), size=(H, W), mode='bilinear', align_corners=False)]
        if return_boundary:
            outs.append(F.interpolate(self.boundary_head(feat), size=(H, W), mode='bilinear', align_corners=False))
        if return_aux and self.training:
            outs.append(F.interpolate(self.aux_head(mid), size=(H, W), mode='bilinear', align_corners=False))

        return outs[0] if len(outs) == 1 else tuple(outs)

# --- Losses ---
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2.0, ignore_index=IGNORE_INDEX):
        super().__init__()
        self.weight = weight; self.gamma = gamma; self.ignore_index = ignore_index
    def forward(self, logits, target):
        ce = F.cross_entropy(logits, target, weight=self.weight, ignore_index=self.ignore_index, reduction='none')
        return (((1 - torch.exp(-ce)) ** self.gamma) * ce).mean()

class ClassBalancedDiceLoss(nn.Module):
    def __init__(self, n_classes=NUM_CLASSES, smooth=1e-5, ignore_index=IGNORE_INDEX, class_weights=None):
        super().__init__()
        self.n_classes = n_classes; self.smooth = smooth; self.ignore_index = ignore_index; self.class_weights = class_weights
    def forward(self, logits, target):
        prob = F.softmax(logits, dim=1)
        mask = (target != self.ignore_index)
        t = target.clone(); t[~mask] = 0
        onehot = F.one_hot(t, self.n_classes).permute(0, 3, 1, 2).float() * mask.unsqueeze(1)
        prob = prob * mask.unsqueeze(1)
        inter = (prob * onehot).sum(dim=(2, 3))
        dice = (2 * inter + self.smooth) / (prob.sum(dim=(2, 3)) + onehot.sum(dim=(2, 3)) + self.smooth)
        if self.class_weights is not None:
            w = self.class_weights.to(dice.device)
            return 1 - (dice.mean(dim=0) * w).sum() / (w.sum() + 1e-12)
        return 1 - dice.mean()

def make_boundary_mask(target, ignore_index=IGNORE_INDEX, dilate=2):
    valid = (target != ignore_index)
    t = target.clone(); t[~valid] = -1
    edge = torch.zeros_like(t, dtype=torch.bool)
    edge[:, 1:] |= (t[:, 1:] != t[:, :-1]) & valid[:, 1:]
    edge[:, :-1] |= (t[:, :-1] != t[:, 1:]) & valid[:, :-1]
    edge[:, :, 1:] |= (t[:, :, 1:] != t[:, :, :-1]) & valid[:, :, 1:]
    edge[:, :, :-1] |= (t[:, :, :-1] != t[:, :, 1:]) & valid[:, :, :-1]
    edge = edge.float().unsqueeze(1)
    for _ in range(max(0, dilate)): edge = F.max_pool2d(edge, kernel_size=3, stride=1, padding=1)
    return (edge > 0).float()

def ohem_cross_entropy(logits, target, weight, ignore_index=IGNORE_INDEX, min_kept=131072, thresh=0.7):
    with torch.no_grad():
        prob = F.softmax(logits, dim=1)
        valid = (target != ignore_index)
        t_safe = target.clone(); t_safe[~valid] = 0
        p_gt = prob.gather(1, t_safe.unsqueeze(1)).squeeze(1)
        hard = (p_gt < thresh) & valid
    loss = F.cross_entropy(logits, target, weight=weight, ignore_index=ignore_index, reduction='none')
    if loss[hard].numel() >= min_kept: return torch.topk(loss[hard], k=min_kept).values.mean()
    return torch.topk(loss[valid], k=min(min_kept, loss[valid].numel())).values.mean()

# --- Base Training Function ---
def tta_predict_logits(model, x, out_hw, img_size, scales=(1.0,), do_flip=True):
    B, C = x.shape[0], NUM_CLASSES
    acc = torch.zeros((B, C, out_hw[0], out_hw[1]), device=x.device)
    n_aug = 0
    for s in scales:
        hs, ws = int(img_size * s), int(img_size * s)
        xs = F.interpolate(x, size=(hs, ws), mode='bilinear', align_corners=False)
        out = F.interpolate(model(xs), size=out_hw, mode='bilinear', align_corners=False)
        acc += F.softmax(out, dim=1); n_aug += 1
        if do_flip:
            xsf = torch.flip(xs, dims=[3])
            out_f = torch.flip(F.interpolate(model(xsf), size=out_hw, mode='bilinear', align_corners=False), dims=[3])
            acc += F.softmax(out_f, dim=1); n_aug += 1
    return acc / n_aug

# ★変更: train_idx, val_idx を受け取り、Subsetでデータを作るように修正
def run_base_train_and_submit(dataset_root, train_idx, val_idx, img_size=768, batch_size=16, epochs=60, lr=1e-4, seed=42, phase1_epochs=40, save_cm_every=5, drive_dir=None, base_best_out="/content/base_best_model.pt"):
    set_seed(seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    run_id = make_run_id("base"); run_dir = ensure_dir(os.path.join("/content", run_id))
    logger = RunLogger(run_dir, run_id)
    base_best = os.path.join(run_dir, "base_best_model.pt")

    print(f"[BASE] run_id: {run_id}, Device: {device}")

    # Data Augmentation
    train_aug = A.Compose([
        A.RandomResizedCrop(size=(img_size, img_size), scale=(0.7, 1.0), ratio=(0.9, 1.1), p=1.0),
        A.HorizontalFlip(p=0.5), A.Rotate(limit=10, p=0.3), A.ColorJitter(p=0.5)
    ], additional_targets={'depth': 'image', 'height': 'image'})
    val_aug = A.Compose([A.Resize(img_size, img_size)], additional_targets={'depth': 'image', 'height': 'image'})

    # ★変更: 受け取ったインデックス(train_idx, val_idx)を使用
    train_loader = DataLoader(
        Subset(NYUv2Dataset(dataset_root, "train", train_aug), train_idx),
        batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True, drop_last=True
    )
    val_loader = DataLoader(
        Subset(NYUv2Dataset(dataset_root, "train", val_aug), val_idx),
        batch_size=8, shuffle=False, num_workers=4, pin_memory=True
    )
    test_loader = DataLoader(
        NYUv2Dataset(dataset_root, "test", val_aug, return_label=False),
        batch_size=1, shuffle=False, num_workers=2, pin_memory=True
    )

    model = ResNeXtDeepLabV3Plus_OS8().to(device)
    ce_weights = torch.tensor([1.0, 12.0, 0.6, 2.2, 0.6, 1.0, 2.0, 1.5, 1.5, 5.0, 3.0, 0.5, 1.0], device=device)

    criterion_focal = FocalLoss(weight=ce_weights)
    criterion_dice = ClassBalancedDiceLoss(class_weights=torch.tensor([1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.2, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0], device=device))
    bce_boundary = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    scaler = GradScaler("cuda")
    best_miou = 0.0

    for epoch in range(1, epochs + 1):
        model.train()
        use_ohem = epoch > phase1_epochs
        w = [0.25, 0.20, 0.00, 0.40, 0.10, 0.05] if not use_ohem else [0.15, 0.00, 0.20, 0.35, 0.10, 0.20] # focal, ce, ohem, dice, aux, bd

        epoch_losses = {"total": 0.0}
        pbar = tqdm(train_loader, desc=f"[BASE] Ep {epoch}")
        for x, y in pbar:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            optimizer.zero_grad()
            with autocast("cuda"):
                seg, bd, aux = model(x, return_aux=True, return_boundary=True)
                loss = (w[0] * criterion_focal(seg, y) + w[1] * (0 if use_ohem else F.cross_entropy(seg, y, weight=ce_weights, ignore_index=IGNORE_INDEX)) +
                        w[2] * (ohem_cross_entropy(seg, y, weight=ce_weights) if use_ohem else 0) + w[3] * criterion_dice(seg, y) +
                        w[4] * F.cross_entropy(aux, y, ignore_index=IGNORE_INDEX) + w[5] * bce_boundary(bd, make_boundary_mask(y)))
            scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
            epoch_losses["total"] += loss.item()
            pbar.set_postfix({"loss": f"{loss.item():.4f}"})

        scheduler.step()

        # Validation
        model.eval()
        cm = torch.zeros((NUM_CLASSES, NUM_CLASSES), dtype=torch.long)
        with torch.no_grad():
            for x, y in val_loader:
                cm = update_cm(cm, model(x.to(device)).argmax(1).cpu(), y.cpu(), NUM_CLASSES, IGNORE_INDEX)
        m = compute_metrics_from_cm(cm)

        save_cm = (epoch % save_cm_every == 0) or (m["miou"] > best_miou)
        logger.log_epoch_val("base", epoch, m, CLASS_NAMES, cm_np=cm.numpy() if save_cm else None)
        fmt_metrics_console("BASE", epoch, m["miou"], m["class_iou"], m["class_precision"], m["class_recall"], CLASS_NAMES)

        if m["miou"] > best_miou:
            best_miou = m["miou"]
            torch.save(model.state_dict(), base_best)
            print(f"  -> New Best mIoU: {best_miou:.5f}")

    # Final & Submit
    print(f"[BASE] Training Finished. Starting Submission...")
    shutil.copy2(base_best, base_best_out)
    model.load_state_dict(torch.load(base_best, map_location=device))
    preds = []
    with torch.no_grad():
        for x, _ in tqdm(test_loader, desc="[BASE] Submit Inference"):
            preds.append(tta_predict_logits(model, x.to(device), (512, 512), img_size, (0.75, 1.0, 1.25, 1.5)).argmax(1).cpu().numpy().astype(np.uint8))

    sub_path = os.path.join(run_dir, "submission.npy")
    np.save(sub_path, np.concatenate(preds, axis=0))
    zip_path = zip_submission_only(sub_path, os.path.join(run_dir, "submission_only.zip"))
    copy_to_drive_if_needed(zip_path, drive_dir)

    return {"run_dir": run_dir, "best_miou": best_miou}

In [22]:
# =========================
# Cell 3: Analysis (Consistent Split)
# =========================
def find_book_neurons_pure(dataset_root, train_idx, top_k=256):
    print("[ANALYSIS] Probing 'Book' neurons using ONLY Train Split...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Feature Extractor (Backbone only)
    full_model = models.resnext101_32x8d(weights=models.ResNeXt101_32X8D_Weights.IMAGENET1K_V1).to(device).eval()
    class BackboneFeature(nn.Module):
        def __init__(self, m): super().__init__(); self.m = m
        def forward(self, x):
            return self.m.layer4(self.m.layer3(self.m.layer2(self.m.layer1(self.m.maxpool(self.m.relu(self.m.bn1(self.m.conv1(x))))))))
    extractor = BackboneFeature(full_model)

    # ★変更: Subsetを使って学習データ(train_idx)のみを対象にする
    ds = Subset(NYUv2Dataset(dataset_root, split="train", transform=None), train_idx)

    act_sum = torch.zeros(2048).to(device)
    count = 0

    with torch.no_grad():
        # Subsetなので range(len(ds)) で回せば train_idx の画像が取れる
        for idx in tqdm(range(len(ds))):
            x, y = ds[idx]
            if BOOK_ID not in y: continue

            rgb = F.interpolate(x[:3].unsqueeze(0).to(device), size=(512, 512))
            mask = F.interpolate((y == BOOK_ID).float().unsqueeze(0).unsqueeze(0).to(device), size=(16, 16), mode='nearest')
            feat = extractor(rgb)

            if mask.sum() > 0:
                act_sum += (feat * mask).sum(dim=(2, 3)).squeeze(0) / mask.sum()
                count += 1

    if count == 0: return list(range(top_k))
    return torch.topk(act_sum / count, k=top_k).indices.tolist()

In [24]:
# =========================
# Cell 4: Boost Model & Training (Consistent Split)
# =========================
class TriDeepLabV3Plus_Boosted(nn.Module):
    def __init__(self, book_indices, in_channels=5, out_classes=13):
        super().__init__()
        self.book_indices = book_indices
        self.backbone = models.resnext101_32x8d(weights=models.ResNeXt101_32X8D_Weights.IMAGENET1K_V1)

        # 5ch Input Adaptation
        old_conv = self.backbone.conv1
        self.backbone.conv1 = nn.Conv2d(in_channels, old_conv.out_channels, 7, 2, 3, bias=False)
        with torch.no_grad():
            self.backbone.conv1.weight[:, :3] = old_conv.weight
            self.backbone.conv1.weight[:, 3:] = old_conv.weight.mean(dim=1, keepdim=True).repeat(1, in_channels-3, 1, 1)

        self.layers = nn.ModuleList([self.backbone.conv1, self.backbone.bn1, self.backbone.relu, self.backbone.maxpool,
                                     self.backbone.layer1, self.backbone.layer2, self.backbone.layer3, self.backbone.layer4])
        self.main_decoder = nn.Sequential(nn.Conv2d(2048, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(), nn.Conv2d(256, out_classes, 1))

        # Boost Branch
        self.book_branch = nn.Sequential(nn.Conv2d(len(book_indices), 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(), nn.Conv2d(64, 1, 1), nn.Sigmoid())

    def forward(self, x):
        input_size = x.shape[-2:]
        feat = x
        for layer in self.layers: feat = layer(feat)

        # Main Path
        main_out = F.interpolate(self.main_decoder(feat), size=input_size, mode='bilinear', align_corners=False)

        # Injection
        boost = F.interpolate(self.book_branch(feat[:, self.book_indices]), size=input_size, mode='bilinear', align_corners=False)
        mask = torch.zeros_like(main_out); mask[:, BOOK_ID] = boost.squeeze(1) * 2.0
        return main_out + mask

# ★変更: train_idx, val_idx を受け取る
def run_tri_train_boost(book_indices, dataset_root, train_idx, val_idx, img_size, batch_size, epochs, lr, seed, save_cm_every, fused_eval_every, fused_tta_light, BASE_MODEL_PATH):
    run_dir = "runs/tri_boost_experiment"; os.makedirs(run_dir, exist_ok=True)
    logger = setup_console_logger(run_dir)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load Frozen Base Model
    base_model = ResNeXtDeepLabV3Plus_OS8().to(device)
    base_model.load_state_dict(torch.load(BASE_MODEL_PATH, map_location=device))
    base_model.eval()

    model = TriDeepLabV3Plus_Boosted(book_indices).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss(ignore_index=IGNORE_INDEX)

    # ★変更: 画像サイズ(img_size)を使ったAugmentationを定義
    train_aug = A.Compose([
        A.RandomResizedCrop(size=(img_size, img_size), scale=(0.8, 1.0), ratio=(0.9, 1.1), p=1.0),
        A.HorizontalFlip(p=0.5),
        A.ColorJitter(p=0.3)
    ], additional_targets={'depth': 'image', 'height': 'image'})

    val_aug = A.Compose([
        A.Resize(img_size, img_size)
    ], additional_targets={'depth': 'image', 'height': 'image'})

    # ★変更: Subsetで受け取ったインデックスを使用
    train_loader = DataLoader(
        Subset(NYUv2Dataset(dataset_root, "train", transform=train_aug), train_idx),
        batch_size=batch_size, shuffle=True, num_workers=4
    )
    val_loader = DataLoader(
        Subset(NYUv2Dataset(dataset_root, "train", transform=val_aug), val_idx),
        batch_size=batch_size, shuffle=False, num_workers=4
    )

    best_miou = 0.0
    best_path = os.path.join(run_dir, "tri_best_model.pt")

    for epoch in range(epochs):
        model.train()
        loss_sum = 0
        pbar = tqdm(train_loader, desc=f"Ep {epoch+1} [Train]")
        for x, y in pbar:
            optimizer.zero_grad()
            out = model(x.to(device))
            loss = criterion(out, y.to(device))
            loss.backward(); optimizer.step()
            loss_sum += loss.item()
            pbar.set_postfix({"loss": f"{loss.item():.4f}"})

        if (epoch + 1) % fused_eval_every == 0:
            model.eval(); base_model.eval()
            iou_meter = IoUMeter(NUM_CLASSES)
            with torch.no_grad():
                for x, y in tqdm(val_loader, desc=f"Ep {epoch+1} [Val]"):
                    x = x.to(device)
                    # Fusion Logic (Cell 1 Unified Function)
                    pred = fuse_conditional_boost(base_model(x), model(x)).argmax(1)
                    iou_meter.update(pred, y)

            m = iou_meter.get_metrics()
            fmt_metrics_console("BOOST", epoch+1, m['miou'], m['iou'], m['precision'], m['recall'], CLASS_NAMES)

            if (epoch + 1) % save_cm_every == 0:
                cm_path = os.path.join(run_dir, f"cm_epoch_{epoch+1}.npy")
                np.save(cm_path, m['cm'])
                print(f"[OUTPUT] Confusion Matrix saved: {cm_path}")

            if m['miou'] > best_miou:
                best_miou = m['miou']
                torch.save(model.state_dict(), best_path)
                logger.info("  -> New Best!")

    return {"tri_best": best_path, "run_dir": run_dir}

In [25]:
# =========================
# Cell 5: Main Execution (Commander of Split)
# =========================
import os
import shutil
import datetime
import pytz
from torch.utils.data import DataLoader
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm
import albumentations as A
import random

def get_jst_timestamp():
    tz_jst = datetime.timezone(datetime.timedelta(hours=9))
    now = datetime.datetime.now(tz_jst)
    return now.strftime("%Y%m%d_%H%M%S_JST")

def backup_run_to_drive(local_run_dir, drive_root, folder_prefix):
    if local_run_dir is None:
        print(f"[ERROR] local_run_dir is None. Backup skipped.")
        return
    if not os.path.exists(local_run_dir):
        print(f"[WARN] Local dir not found: {local_run_dir}")
        return

    timestamp = get_jst_timestamp()
    folder_name = f"{folder_prefix}_{timestamp}"
    drive_dest_path = os.path.join(drive_root, folder_name)

    print(f"\n[BACKUP] Copying results to Drive...")
    print(f"  Src:  {local_run_dir}")
    print(f"  Dest: {drive_dest_path}")

    try:
        if os.path.exists(drive_dest_path): shutil.rmtree(drive_dest_path)
        shutil.copytree(local_run_dir, drive_dest_path)
        print(f"[BACKUP] Success! Saved to: {drive_dest_path}")
    except Exception as e:
        print(f"[ERROR] Backup failed: {e}")

def run_fused13_submit_for_boosted(dataset_root, img_size, BASE_MODEL_PATH, TRI_MODEL_PATH, book_indices, out_dir):
    ensure_dir(out_dir)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n[SUBMIT] Generating 13-Class Fused submission...")

    base_model = ResNeXtDeepLabV3Plus_OS8().to(device).eval()
    base_model.load_state_dict(torch.load(BASE_MODEL_PATH, map_location=device))

    tri_model = TriDeepLabV3Plus_Boosted(book_indices).to(device).eval()
    tri_model.load_state_dict(torch.load(TRI_MODEL_PATH, map_location=device))

    test_loader = DataLoader(
        NYUv2Dataset(dataset_root, "test", A.Compose([A.Resize(img_size, img_size)], additional_targets={'depth':'image','height':'image'}), return_label=False),
        batch_size=1, shuffle=False, num_workers=2
    )

    preds = []
    with torch.no_grad():
        for x, _ in tqdm(test_loader, desc="[SUBMIT] Inference"):
            x = x.to(device)
            b_sum = torch.zeros(1, NUM_CLASSES, 512, 512).to(device)
            t_sum = torch.zeros(1, NUM_CLASSES, 512, 512).to(device)
            for scale in [0.5, 0.75, 1.0, 1.25, 1.5]:
                h, w = int(512*scale), int(512*scale)
                xs = F.interpolate(x, size=(h, w), mode='bilinear', align_corners=False)
                b_out = base_model(xs)
                b_sum += F.interpolate(b_out, size=(512,512), mode='bilinear')
                t_out = tri_model(xs)
                t_sum += F.interpolate(t_out, size=(512,512), mode='bilinear')
                # Flip
                xsf = torch.flip(xs, [3])
                b_sum += torch.flip(F.interpolate(base_model(xsf), size=(512,512), mode='bilinear'), [3])
                t_sum += torch.flip(F.interpolate(tri_model(xsf), size=(512,512), mode='bilinear'), [3])

            pred = fuse_conditional_boost(b_sum, t_sum).argmax(1).cpu().numpy().astype(np.uint8)
            preds.append(pred)

    path = os.path.join(out_dir, "submission.npy")
    np.save(path, np.concatenate(preds, axis=0))
    zip_path = zip_submission_only(path, os.path.join(out_dir, "submission_only.zip"))
    return out_dir

def main():
    mount_drive()
    DRIVE_ROOT = "/content/drive/MyDrive/nyu_runs"
    ensure_dir(DRIVE_ROOT)

    # ★重要: ここで "both" を指定し、BASEから作り直してください
    RUN_MODE = "both"

    print(f"==========================================")
    print(f" START PIPELINE at {get_jst_timestamp()}")
    print(f" MODE: {RUN_MODE}")
    print(f"==========================================\n")

    base_model_path = "/content/base_best_model.pt"

    # --- 重要: データ分割をここで確定させ、BASE/BOOSTで共有する ---
    full_dataset = NYUv2Dataset("/content/data", split="train", transform=None)
    n_total = len(full_dataset)
    indices = list(range(n_total))

    # ★SEED 42 で固定シャッフル
    random.seed(42)
    random.shuffle(indices)

    n_val = int(n_total * 0.1)
    train_idx = indices[:-n_val]
    val_idx = indices[-n_val:]

    print(f"Dataset Split: Total={n_total}, Train={len(train_idx)}, Val={len(val_idx)}")
    # 検証用: 先頭5件を表示して、次回実行時と同じか確認できるようにする
    print(f"  (Debug) First 5 Train Indices: {train_idx[:5]}")
    print(f"  (Debug) First 5 Val Indices:   {val_idx[:5]}")

    # --- BASE MODEL ---
    if RUN_MODE in ["base", "both"]:
        print("\n>>> Starting BASE Model Training...")
        # ★修正: train_idx, val_idx を渡す
        base_res = run_base_train_and_submit(
            dataset_root="/content/data",
            train_idx=train_idx,
            val_idx=val_idx,
            img_size=768, batch_size=16, epochs=60, lr=1e-4,
            save_cm_every=5, drive_dir=None, base_best_out=base_model_path
        )

        if base_res is None:
            print("[CRITICAL ERROR] run_base_train_and_submit returned None!")
            return

        backup_run_to_drive(base_res["run_dir"], DRIVE_ROOT, "BASE")

    # --- BOOST MODEL ---
    if RUN_MODE in ["tri", "both"]:
        if not os.path.exists(base_model_path):
            print(f"[ERROR] Base model not found at {base_model_path}.")
            return

        print("\n>>> Starting BOOST Model Training...")
        try:
            # ★修正: train_idx を渡す
            indices_book = find_book_neurons_pure("/content/data", train_idx=train_idx)
        except:
            print("[WARN] Analysis failed. Using dummy indices.")
            indices_book = list(range(256))

        # ★修正: train_idx, val_idx を渡す
        tri_res = run_tri_train_boost(
            book_indices=indices_book,
            dataset_root="/content/data",
            train_idx=train_idx,
            val_idx=val_idx,
            img_size=768, batch_size=16, epochs=25, lr=3e-4, seed=42,
            save_cm_every=5, fused_eval_every=1, fused_tta_light=True, BASE_MODEL_PATH=base_model_path
        )

        if tri_res is None:
            print("[CRITICAL ERROR] run_tri_train_boost returned None!")
            return

        submit_dir = os.path.join(tri_res["run_dir"], "submit_fused")
        run_fused13_submit_for_boosted(
            dataset_root="/content/data", img_size=768, BASE_MODEL_PATH=base_model_path,
            TRI_MODEL_PATH=tri_res["tri_best"], book_indices=indices_book, out_dir=submit_dir
        )

        backup_run_to_drive(tri_res["run_dir"], DRIVE_ROOT, "BOOST")

    print(f"\n==========================================")
    print(f" ALL DONE at {get_jst_timestamp()}")
    print(f"==========================================")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 START PIPELINE at 20260106_125108_JST
 MODE: both

Dataset Split: Total=795, Train=716, Val=79
  (Debug) First 5 Train Indices: [578, 651, 749, 307, 602]
  (Debug) First 5 Val Indices:   [373, 650, 464, 786, 748]

>>> Starting BASE Model Training...
[BASE] run_id: base_20260106_035108, Device: cuda


[BASE] Ep 1: 100%|██████████| 44/44 [00:59<00:00,  1.34s/it, loss=0.9854]


[BASE] Epoch 01 | mIoU=0.34924
Class           IoU     Prec      Rec
-------------------------------------
Bed           0.512    0.718    0.641
Book          0.107    0.113    0.675
Ceiling       0.018    0.617    0.018
Chair         0.411    0.560    0.608
Floor         0.712    0.859    0.806
Cabinet       0.248    0.553    0.311
Object        0.403    0.469    0.742
Picture       0.336    0.556    0.459
Sofa          0.308    0.562    0.406
Desk          0.208    0.234    0.653
TV            0.230    0.601    0.271
Wall          0.594    0.885    0.644
Window        0.453    0.684    0.572
  -> New Best mIoU: 0.34924


[BASE] Ep 2: 100%|██████████| 44/44 [00:58<00:00,  1.34s/it, loss=0.8518]


[BASE] Epoch 02 | mIoU=0.40906
Class           IoU     Prec      Rec
-------------------------------------
Bed           0.431    0.761    0.499
Book          0.151    0.162    0.680
Ceiling       0.327    0.656    0.395
Chair         0.391    0.687    0.476
Floor         0.775    0.974    0.792
Cabinet       0.364    0.616    0.471
Object        0.383    0.412    0.848
Picture       0.451    0.595    0.650
Sofa          0.391    0.581    0.545
Desk          0.282    0.444    0.436
TV            0.278    0.351    0.574
Wall          0.635    0.942    0.661
Window        0.457    0.662    0.596
  -> New Best mIoU: 0.40906


...

[BOOST] Epoch 24 | mIoU=0.66772
Class           IoU     Prec      Rec
-------------------------------------
Bed           0.780    0.864    0.889
Book          0.203    0.224    0.684
Ceiling       0.721    0.845    0.832
Chair         0.703    0.838    0.814
Floor         0.940    0.975    0.963
Cabinet       0.594    0.770    0.723
Object        0.586    0.704    0.778
Picture       0.670    0.873    0.743
Sofa          0.647    0.847    0.733
Desk          0.466    0.656    0.617
TV            0.742    0.981    0.753
Wall          0.881    0.949    0.924
Window        0.745    0.863    0.845


Ep 25 [Val]: 100%|██████████| 5/5 [00:06<00:00,  1.30s/it]


[BOOST] Epoch 25 | mIoU=0.66729
Class           IoU     Prec      Rec
-------------------------------------
Bed           0.780    0.864    0.889
Book          0.200    0.220    0.691
Ceiling       0.721    0.845    0.832
Chair         0.703    0.838    0.814
Floor         0.940    0.975    0.963
Cabinet       0.592    0.770    0.719
Object        0.586    0.704    0.777
Picture       0.670    0.873    0.743
Sofa          0.647    0.847    0.733
Desk          0.466    0.656    0.617
TV            0.742    0.981    0.753
Wall          0.881    0.949    0.924
Window        0.745    0.863    0.845
[OUTPUT] Confusion Matrix saved: runs/tri_boost_experiment/cm_epoch_25.npy

[SUBMIT] Generating 13-Class Fused submission...


[SUBMIT] Inference: 100%|██████████| 654/654 [04:03<00:00,  2.68it/s]


[LONG] Start zipping submission only: runs/tri_boost_experiment/submit_fused/submission_only.zip
[DONE] submission zip: runs/tri_boost_experiment/submit_fused/submission_only.zip

[BACKUP] Copying results to Drive...
  Src:  runs/tri_boost_experiment
  Dest: /content/drive/MyDrive/nyu_runs/BOOST_20260106_142805_JST
[BACKUP] Success! Saved to: /content/drive/MyDrive/nyu_runs/BOOST_20260106_142805_JST

 ALL DONE at 20260106_142806_JST


In [ ]:
# =========================
# Check Base Model Performance Alone
# =========================
def check_base_performance(dataset_root, model_path, img_size=768):
    print(f"[CHECK] Evaluating Base Model Alone: {model_path}")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load Base Model
    base_model = ResNeXtDeepLabV3Plus_OS8(num_classes=13, in_channels=5).to(device)
    base_model.load_state_dict(torch.load(model_path, map_location=device))
    base_model.eval()

    # Dataset & Loader
    val_ds = NYUv2Dataset(dataset_root, split="val", transform=None)
    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=4)

    iou_meter = IoUMeter(num_classes=13)

    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="Base Eval"):
            x, y = x.to(device), y.to(device).long()

            # Predict
            # 画像サイズに合わせてリサイズが必要ならここに挟む
            # ここでは簡易的にそのまま流す（学習時と同じ前提）
            out = base_model(x)
            pred = out.argmax(dim=1)
            iou_meter.update(pred, y)

    metrics = iou_meter.get_metrics()
    print(f"\n[Base Model Result]")
    print(f"mIoU: {metrics['miou']:.4f}")
    print(f"Book IoU: {metrics['iou'][1]:.4f}") # Book ID=1
    print("------------------------------------------------")

# 実行
check_base_performance("/content/data", "/content/base_best_model.pt")

[CHECK] Evaluating Base Model Alone: /content/base_best_model.pt


Base Eval: 100%|██████████| 795/795 [00:26<00:00, 29.89it/s]


[Base Model Result]
mIoU: 0.8627
Book IoU: 0.6601
------------------------------------------------


In [ ]:
import json
import matplotlib.pyplot as plt
import glob
import os

def plot_learning_curves(log_file):
    epochs = []
    losses = []
    mious = []
    lrs = []

    # ログファイルの読み込み
    with open(log_file, 'r') as f:
        for line in f:
            data = json.loads(line)
            # ヘッダー行（info）はスキップ
            if "info" in data:
                continue

            epochs.append(data['epoch'])
            losses.append(data['loss'])
            mious.append(data['val_miou'])
            lrs.append(data['lr'])

    # プロットの作成
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Loss (左軸・赤)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Training Loss', color='tab:red')
    ax1.plot(epochs, losses, color='tab:red', label='Train Loss', marker='.')
    ax1.tick_params(axis='y', labelcolor='tab:red')
    ax1.grid(True, alpha=0.3)

    # mIoU (右軸・青)
    ax2 = ax1.twinx()
    ax2.set_ylabel('Validation mIoU', color='tab:blue')
    ax2.plot(epochs, mious, color='tab:blue', label='Val mIoU', marker='.')
    ax2.tick_params(axis='y', labelcolor='tab:blue')

    plt.title(f'Learning Curve: {os.path.basename(log_file)}')
    fig.tight_layout()
    plt.show()

    # 学習率の確認用プロット（Poly Schedulerが効いているか確認）
    plt.figure(figsize=(12, 3))
    plt.plot(epochs, lrs, color='orange')
    plt.title("Learning Rate Schedule")
    plt.xlabel("Epoch")
    plt.ylabel("LR")
    plt.grid(True, alpha=0.3)
    plt.show()

# 自動で最新のログファイルを見つけて描画
log_files = sorted(glob.glob("train_log_detailed_*.jsonl"))
if log_files:
    latest_log = log_files[-1]
    print(f"Plotting: {latest_log}")
    plot_learning_curves(latest_log)
else:
    print("No log file found.")

In [ ]:
# -*- coding: utf-8 -*-
# NYUv2 val visualization (RGB | GT | Pred | RGB+Error) with FIXED palette -> ZIP
# - NO matplotlib colormap
# - GT and Pred share EXACT same palette mapping (ID -> RGB)
# - ignore_index(255) is shown as black by default

import os
import zipfile
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Subset
from torchvision.datasets import VisionDataset
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, InterpolationMode
from torchvision import models

# =========================
# EDIT HERE
# =========================
DATASET_ROOT = "/content/data"
IMAGE_SIZE = (512, 512)

MODEL_PATH = "/content/checkpoints/model_20260103062841.pt"  # ←ここだけ直す
OUTDIR = "val_viz_fixed"
ZIP_PATH = "val_viz_fixed.zip"

MAX_SAVE = 50  # Noneで全件
SEED = 42
TRAIN_VAL_SPLIT = 0.9

NUM_WORKERS = 0
BATCH_SIZE = 1

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IGNORE_INDEX = 255

print("DEVICE:", DEVICE)
print("MODEL_PATH:", MODEL_PATH)

# =========================
# FIXED PALETTE (ID -> RGB)
# ここが「色の意味」そのもの。GTもPredも必ずこれを通す。
# =========================
PALETTE_13 = np.array([
    [  0,   0,   0],   # 0 wall
    [128,  64,  32],   # 1 floor
    [  0, 128, 255],   # 2 cabinet
    [255,   0, 128],   # 3 bed
    [  0, 255,   0],   # 4 chair
    [255, 128,   0],   # 5 sofa
    [255, 255,   0],   # 6 table
    [128, 128, 128],   # 7 door
    [  0, 255, 255],   # 8 window
    [  0,  64, 128],   # 9 bookshelf
    [255,   0,   0],   # 10 picture
    [128,   0, 255],   # 11 counter
    [ 64, 255, 128],   # 12 desk
], dtype=np.uint8)

# =========================
# Dataset helpers
# =========================
def pil_label_to_long_tensor(lbl_pil: Image.Image) -> torch.Tensor:
    arr = np.array(lbl_pil, dtype=np.int64)
    return torch.from_numpy(arr).long()

def depth_pil_to_tensor_01(depth_pil: Image.Image, size_hw):
    depth_pil = depth_pil.resize((size_hw[1], size_hw[0]), resample=Image.BILINEAR)
    arr = np.array(depth_pil)
    if arr.dtype == np.uint16 or (arr.max() > 255):
        arr = arr.astype(np.float32) / 65535.0
    else:
        arr = arr.astype(np.float32) / 255.0
    arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)
    arr = np.clip(arr, 0.0, 1.0)
    return torch.from_numpy(arr).unsqueeze(0)  # [1,H,W]

class NYUv2(VisionDataset):
    def __init__(self, root, split="train", include_depth=True,
                 image_transform=None, depth_transform=None, target_transform=None):
        super().__init__(root)
        assert split in ("train", "test")
        self.root = root
        self.split = split
        self.include_depth = include_depth

        images_dir = os.path.join(self.root, self.split, "image")
        img_names = sorted(os.listdir(images_dir))

        self.images = [os.path.join(images_dir, n) for n in img_names]
        self.depths = [os.path.join(self.root, self.split, "depth", n) for n in img_names]

        if self.split == "train":
            self.targets = [os.path.join(self.root, self.split, "label", n) for n in img_names]
        else:
            self.targets = None

        self.image_transform = image_transform
        self.depth_transform = depth_transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert("RGB")
        depth = Image.open(self.depths[idx])

        if self.image_transform is not None:
            image = self.image_transform(image)
        if self.depth_transform is not None:
            depth = self.depth_transform(depth)

        if self.split == "test":
            return (image, depth) if self.include_depth else image

        target = Image.open(self.targets[idx])
        if self.target_transform is not None:
            target = self.target_transform(target)

        return (image, depth, target) if self.include_depth else (image, target)

# =========================
# Model blocks (ResNet50-UNet + optional SE)
# =========================
class ConvBNReLU(nn.Module):
    def __init__(self, in_ch, out_ch, k=3, p=1):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=k, padding=p, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.block(x)

class UpBlock(nn.Module):
    def __init__(self, in_ch, skip_ch, out_ch):
        super().__init__()
        self.conv1 = ConvBNReLU(in_ch + skip_ch, out_ch)
        self.conv2 = ConvBNReLU(out_ch, out_ch)

    def forward(self, x, skip):
        x = nn.functional.interpolate(x, size=skip.shape[-2:], mode="bilinear", align_corners=False)
        x = torch.cat([x, skip], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x

class DepthStem(nn.Module):
    def __init__(self, z_ch=64):
        super().__init__()
        self.net = nn.Sequential(
            ConvBNReLU(1, 32, 3, 1),
            nn.MaxPool2d(2),
            ConvBNReLU(32, 48, 3, 1),
            nn.MaxPool2d(2),
            ConvBNReLU(48, z_ch, 3, 1),
            nn.MaxPool2d(2),
        )
    def forward(self, d):
        return self.net(d)

class SEFromDepth(nn.Module):
    def __init__(self, feat_ch, z_ch, reduction=16, alpha=0.05):
        super().__init__()
        self.alpha = float(alpha)
        hidden = max(32, feat_ch // reduction)
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(z_ch, hidden, 1, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, feat_ch, 1, bias=True),
            nn.Sigmoid(),
        )
    def forward(self, F, z):
        g = self.fc(z)  # [B,C,1,1]
        scale = 1.0 + self.alpha * (g - 0.5)
        return F * scale

class ResNet50UNet(nn.Module):
    def __init__(self, num_classes=13, coarse_classes=5, in_channels=4,
                 pretrained=False,
                 use_se=False, se_z_ch=64, se_reduction=16, se_alpha=0.05,
                 se_dec_stages=("up1",)):
        super().__init__()
        self.use_se = use_se
        self.se_dec_stages = tuple(se_dec_stages)

        resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2 if pretrained else None)

        old_conv = resnet.conv1
        if in_channels != old_conv.in_channels:
            new_conv = nn.Conv2d(
                in_channels, old_conv.out_channels,
                kernel_size=old_conv.kernel_size,
                stride=old_conv.stride,
                padding=old_conv.padding,
                bias=False
            )
            with torch.no_grad():
                new_conv.weight[:, :3] = old_conv.weight
                mean_w = old_conv.weight.mean(dim=1, keepdim=True)
                for c in range(3, in_channels):
                    new_conv.weight[:, c:c+1] = mean_w
            resnet.conv1 = new_conv

        self.enc0 = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu)  # 1/2
        self.pool = resnet.maxpool                                        # 1/4
        self.enc1 = resnet.layer1                                         # 1/4
        self.enc2 = resnet.layer2                                         # 1/8
        self.enc3 = resnet.layer3                                         # 1/16
        self.enc4 = resnet.layer4                                         # 1/32

        self.center = ConvBNReLU(2048, 1024)
        self.up4 = UpBlock(1024, 1024, 512)
        self.up3 = UpBlock(512,  512,  256)
        self.up2 = UpBlock(256,  256,  128)
        self.up1 = UpBlock(128,  64,   64)

        self.head_feat = ConvBNReLU(64, 64)
        self.head_out  = nn.Conv2d(64, num_classes, kernel_size=1)

        self.coarse_head = nn.Conv2d(256, coarse_classes, kernel_size=1)
        self.boundary_head = nn.Sequential(ConvBNReLU(64, 32), nn.Conv2d(32, 1, 1))

        if self.use_se:
            self.depth_stem = DepthStem(z_ch=se_z_ch)
            stage_to_ch = {"center":1024, "up4":512, "up3":256, "up2":128, "up1":64}
            self.se_dec = nn.ModuleDict()
            for st in self.se_dec_stages:
                self.se_dec[st] = SEFromDepth(stage_to_ch[st], se_z_ch, se_reduction, se_alpha)
        else:
            self.depth_stem = None
            self.se_dec = None

    def _apply_dec_se(self, feat, z, stage_name: str):
        if (not self.use_se) or (self.se_dec is None) or (stage_name not in self.se_dec):
            return feat
        z_ = nn.functional.interpolate(z, size=feat.shape[-2:], mode="bilinear", align_corners=False)
        return self.se_dec[stage_name](feat, z_)

    def forward(self, x, depth=None):
        c1 = self.enc0(x)
        t  = self.pool(c1)
        c2 = self.enc1(t)
        c3 = self.enc2(c2)
        c4 = self.enc3(c3)
        c5 = self.enc4(c4)

        z = self.depth_stem(depth) if self.use_se else None

        x = self.center(c5)
        if z is not None: x = self._apply_dec_se(x, z, "center")
        x = self.up4(x, c4)
        if z is not None: x = self._apply_dec_se(x, z, "up4")
        x = self.up3(x, c3)
        if z is not None: x = self._apply_dec_se(x, z, "up3")

        coarse_logits = self.coarse_head(x)

        x = self.up2(x, c2)
        if z is not None: x = self._apply_dec_se(x, z, "up2")
        x = self.up1(x, c1)
        if z is not None: x = self._apply_dec_se(x, z, "up1")

        x = nn.functional.interpolate(x, scale_factor=2, mode="bilinear", align_corners=False)
        feat = self.head_feat(x)
        fine_logits = self.head_out(feat)
        boundary_logit = self.boundary_head(x)
        return fine_logits, coarse_logits, boundary_logit

# =========================
# FIXED colorization (ID -> RGB)
# =========================
def id_to_rgb(mask_hw: np.ndarray, ignore_index=255, ignore_rgb=(0,0,0)) -> np.ndarray:
    """
    mask_hw: [H,W] int (0..12 or 255)
    returns: [H,W,3] uint8
    """
    out = np.zeros((mask_hw.shape[0], mask_hw.shape[1], 3), dtype=np.uint8)
    ign = (mask_hw == ignore_index)
    valid = ~ign
    m = mask_hw.copy()
    m[ign] = 0
    m = np.clip(m, 0, 12)
    out[valid] = PALETTE_13[m[valid]]
    out[ign] = np.array(ignore_rgb, dtype=np.uint8)
    return out

def denorm_rgb(rgb_tensor: torch.Tensor) -> Image.Image:
    mean = torch.tensor([0.485, 0.456, 0.406], device=rgb_tensor.device).view(3,1,1)
    std  = torch.tensor([0.229, 0.224, 0.225], device=rgb_tensor.device).view(3,1,1)
    x = (rgb_tensor * std + mean).clamp(0, 1)
    x = (x * 255).byte().permute(1,2,0).cpu().numpy()
    return Image.fromarray(x, mode="RGB")

def overlay_error(rgb_pil: Image.Image, gt_hw: np.ndarray, pred_hw: np.ndarray, ignore_index=255) -> Image.Image:
    rgb = np.array(rgb_pil).astype(np.float32)
    err = (gt_hw != ignore_index) & (gt_hw != pred_hw)
    if err.any():
        overlay = rgb.copy()
        overlay[err] = 0.6 * overlay[err] + 0.4 * np.array([255, 0, 0], dtype=np.float32)
        overlay = np.clip(overlay, 0, 255).astype(np.uint8)
        return Image.fromarray(overlay, mode="RGB")
    return rgb_pil

def stack_h(images):
    widths = [im.width for im in images]
    heights = [im.height for im in images]
    out = Image.new("RGB", (sum(widths), max(heights)))
    x = 0
    for im in images:
        out.paste(im, (x, 0))
        x += im.width
    return out

# =========================
# Transforms (same as eval)
# =========================
eval_image_transform = Compose([
    Resize(IMAGE_SIZE, interpolation=InterpolationMode.BILINEAR),
    ToTensor(),
    Normalize((0.485,0.456,0.406), (0.229,0.224,0.225)),
])

def depth_transform(pil):
    return depth_pil_to_tensor_01(pil, IMAGE_SIZE)

def target_transform(pil):
    pil = pil.resize((IMAGE_SIZE[1], IMAGE_SIZE[0]), resample=Image.NEAREST)
    return pil_label_to_long_tensor(pil)

# =========================
# Build val split (same indices rule)
# =========================
split_base = NYUv2(
    root=DATASET_ROOT,
    split="train",
    include_depth=True,
    image_transform=eval_image_transform,
    depth_transform=depth_transform,
    target_transform=target_transform,
)

n_total = len(split_base)
n_train = int(n_total * TRAIN_VAL_SPLIT)
n_val = n_total - n_train
g = torch.Generator().manual_seed(SEED)
_, val_subset = random_split(split_base, [n_train, n_val], generator=g)
val_ds = Subset(split_base, val_subset.indices)

val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

print(f"Val samples: {len(val_ds)}")

# =========================
# Load checkpoint + auto-detect SE
# =========================
ckpt = torch.load(MODEL_PATH, map_location=DEVICE)
if isinstance(ckpt, dict) and "state_dict" in ckpt and isinstance(ckpt["state_dict"], dict):
    sd = ckpt["state_dict"]
else:
    sd = ckpt

has_se = any(k.startswith("depth_stem.") or k.startswith("se_dec.") for k in sd.keys())
print("Detected SE in checkpoint:", has_se)

model = ResNet50UNet(
    num_classes=13,
    coarse_classes=5,
    in_channels=4,
    pretrained=False,
    use_se=has_se,
    se_z_ch=64,
    se_reduction=16,
    se_alpha=0.05,
    se_dec_stages=("up1",),
).to(DEVICE)

model.load_state_dict(sd, strict=True)
model.eval()

# =========================
# Generate panels with FIXED palette
# =========================
os.makedirs(OUTDIR, exist_ok=True)

saved = 0
with torch.no_grad():
    for i, (rgb, depth, label) in enumerate(tqdm(val_loader, desc="Saving fixed-color panels")):
        rgb = rgb.to(DEVICE, non_blocking=True)
        depth = depth.to(DEVICE, non_blocking=True)
        label = label.to(DEVICE, non_blocking=True)

        x = torch.cat([rgb.float(), depth.float()], dim=1)
        fine_logits, _, _ = model(x, depth=depth.float() if has_se else None)
        pred = fine_logits.argmax(dim=1)

        rgb_pil = denorm_rgb(rgb[0])

        gt_hw = label[0].cpu().numpy().astype(np.int32)
        pred_hw = pred[0].cpu().numpy().astype(np.int32)

        gt_rgb = id_to_rgb(gt_hw, ignore_index=IGNORE_INDEX, ignore_rgb=(0,0,0))
        pr_rgb = id_to_rgb(pred_hw, ignore_index=IGNORE_INDEX, ignore_rgb=(0,0,0))

        gt_pil = Image.fromarray(gt_rgb, mode="RGB")
        pr_pil = Image.fromarray(pr_rgb, mode="RGB")
        err_pil = overlay_error(rgb_pil, gt_hw, pred_hw, ignore_index=IGNORE_INDEX)

        panel = stack_h([rgb_pil, gt_pil, pr_pil, err_pil])
        panel.save(os.path.join(OUTDIR, f"{i:06d}.png"), optimize=True)

        saved += 1
        if MAX_SAVE is not None and saved >= MAX_SAVE:
            break

print(f"Saved: {saved} images -> {OUTDIR}")

# =========================
# Zip
# =========================
with zipfile.ZipFile(ZIP_PATH, "w", compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zf:
    for fn in sorted(os.listdir(OUTDIR)):
        if fn.lower().endswith(".png"):
            zf.write(os.path.join(OUTDIR, fn), arcname=fn)

print(f"ZIP created: {ZIP_PATH}")


In [ ]:
%ls -l checkpoints

In [ ]:
# ------------------
#    Evaluation (DeepLabV3-ResNet50 / ResNet backbone)
# ------------------

ckpt = torch.load(final_path, map_location=device)

# もし「学習で checkpoint dict（model_state入り）」で保存している場合にも対応
if isinstance(ckpt, dict) and "model_state" in ckpt:
    model.load_state_dict(ckpt["model_state"])
else:
    model.load_state_dict(ckpt)

model.eval()

predictions = []

with torch.no_grad():
    print("Generating predictions...")
    for image, depth in tqdm(test_data):
        image = image.to(device, non_blocking=True)
        depth = depth.to(device, non_blocking=True)

        # 念のため：depthが3ch事故のときは1chに落とす
        if depth.dim() == 4 and depth.size(1) == 3:
            depth = depth[:, :1, :, :]

        x = torch.cat((image, depth), dim=1)   # [B,4,H,W]

        # DeepLabV3 は dict を返すので "out" を使う
        out = model(x)["out"]                  # [B,num_classes,H,W]
        pred = out.argmax(dim=1)               # [B,H,W]

        predictions.append(pred.cpu())

predictions = torch.cat(predictions, dim=0).numpy()
np.save("submission.npy", predictions)
print("Predictions saved to submission.npy")


"""## 提出方法

以下の3点をzip化し，Omnicampusの「最終課題 (セグメンテーション)」から提出してください．

- `submission.npy`
- `model.pt`や`model_best.pt`など，テストに使用した重み（拡張子は`.pt`のみ）
- 本Colab Notebook
"""

from zipfile import ZipFile, ZIP_DEFLATED

notebook_path = "/content/drive/MyDrive/Colab Notebooks/DL_Basic_2025_Competition_NYUv2_baseline.ipynb"

from zipfile import ZipFile, ZIP_DEFLATED

notebook_path = "/content/drive/MyDrive/code.ipynb"

with ZipFile("submission.zip",
             mode="w",
             compression=ZIP_DEFLATED,
             compresslevel=9) as zf:
    zf.write("submission.npy")
    zf.write(model_path)
#    zf.write(notebook_path,
#             arcname="code.ipynb")
